In [1]:
from pyspark.sql import SparkSession
from 

In [2]:
spark=(
    SparkSession
    .builder
    .appName("apache-iceberg")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b08b5770-5512-4f8c-8e4f-ed1133ae99ed;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.3 in central
:: resolution report :: resolve 49ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submi

In [3]:
spark

In [4]:
# DB생성
# 카탈로그 이름(my_iceberg)을 붙여서 DB 생성
spark.sql("CREATE DATABASE IF NOT EXISTS my_iceberg.playground")

DataFrame[]

In [8]:
spark.sql("""
CREATE TABLE IF NOT EXISTS my_iceberg.playground.sample_table(
    id BIGINT,
    data STRING
)
USING iceberg
""")

DataFrame[]

In [11]:
data=[
    (1,"Hello World"),
    (2,"Apache Iceberg"),
    (3, "PySpark Demo")
]
df=spark.createDataFrame(data,["id","data"])
df.writeTo("my_iceberg.playground.sample_table").append()

In [14]:
# spark.sql("SELECT * FROM my_iceberg.playground.sample_table").show()

+---+--------------+
| id|          data|
+---+--------------+
|  1|   Hello World|
|  2|Apache Iceberg|
|  3|  PySpark Demo|
+---+--------------+



In [15]:
spark.sql("USE my_iceberg")

DataFrame[]

In [16]:
spark.sql("SELECT * FROM playground.sample_table").show()

+---+--------------+
| id|          data|
+---+--------------+
|  1|   Hello World|
|  2|Apache Iceberg|
|  3|  PySpark Demo|
+---+--------------+



In [18]:
iceberg_df=spark.read.format("iceberg").table("playground.sample_table")
iceberg_df.show()

+---+--------------+
| id|          data|
+---+--------------+
|  1|   Hello World|
|  2|Apache Iceberg|
|  3|  PySpark Demo|
+---+--------------+



In [19]:
spark.sql("ALTER TABLE playground.sample_table ADD COLUMN extra_info STRING")

DataFrame[]

In [20]:
data_new = [
    (4, "Another record", "This is extra info"),
    (5, "Yet another record", "Additional details")
]
df_new = spark.createDataFrame(data_new, ["id", "data", "extra_info"])
df_new.writeTo("playground.sample_table").append()

In [21]:
spark.sql("SELECT * FROM playground.sample_table").show()

+---+------------------+------------------+
| id|              data|        extra_info|
+---+------------------+------------------+
|  1|       Hello World|              NULL|
|  2|    Apache Iceberg|              NULL|
|  3|      PySpark Demo|              NULL|
|  4|    Another record|This is extra info|
|  5|Yet another record|Additional details|
+---+------------------+------------------+



In [29]:
spark.sql("SELECT * FROM playground.sample_table.snapshots").show(truncate=False,vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 committed_at  | 2026-02-05 05:41:03.225                                                                                                                                                                                                                                                                              
 snapshot_id   | 918297713420570779                                                                                                                                                                                                                                                                                   
 parent_id     | NULL                                              

In [31]:
spark.sql("SELECT * FROM playground.sample_table.history").show(truncate=False)

+-----------------------+------------------+------------------+-------------------+
|made_current_at        |snapshot_id       |parent_id         |is_current_ancestor|
+-----------------------+------------------+------------------+-------------------+
|2026-02-05 05:41:03.225|918297713420570779|NULL              |true               |
|2026-02-05 05:47:18.637|238473521353522660|918297713420570779|true               |
+-----------------------+------------------+------------------+-------------------+



In [33]:
spark.sql("CALL system.rollback_to_snapshot('playground.sample_table',918297713420570779)").show()

+--------------------+-------------------+
|previous_snapshot_id|current_snapshot_id|
+--------------------+-------------------+
|  238473521353522660| 918297713420570779|
+--------------------+-------------------+



In [34]:
spark.sql("SELECT * FROM playground.sample_table").show()

+---+--------------+----------+
| id|          data|extra_info|
+---+--------------+----------+
|  1|   Hello World|      NULL|
|  2|Apache Iceberg|      NULL|
|  3|  PySpark Demo|      NULL|
+---+--------------+----------+



In [40]:
# history 테이블을 조회해서 언제 어떤 ID가 생성됐는지 확인합니다.
spark.sql("SELECT * FROM playground.sample_table.history").show(truncate=False)

+-----------------------+------------------+------------------+-------------------+
|made_current_at        |snapshot_id       |parent_id         |is_current_ancestor|
+-----------------------+------------------+------------------+-------------------+
|2026-02-05 05:41:03.225|918297713420570779|NULL              |true               |
|2026-02-05 05:47:18.637|238473521353522660|918297713420570779|false              |
|2026-02-05 05:55:12.496|918297713420570779|NULL              |true               |
+-----------------------+------------------+------------------+-------------------+



In [43]:
target_id = 238473521353522660
spark.sql(f"CALL system.cherrypick_snapshot('playground.sample_table', {target_id})")

DataFrame[source_snapshot_id: bigint, current_snapshot_id: bigint]

In [44]:
spark.sql("SELECT * FROM playground.sample_table").show()

+---+------------------+------------------+
| id|              data|        extra_info|
+---+------------------+------------------+
|  4|    Another record|This is extra info|
|  5|Yet another record|Additional details|
|  1|       Hello World|              NULL|
|  2|    Apache Iceberg|              NULL|
|  3|      PySpark Demo|              NULL|
+---+------------------+------------------+

